# A study of SDE and stoch processes

Questions that need answering:
* what is the space setup here? bounded continuous function of $[0, T]$? or is it the dual of that (some good behaving measures)? What is the dual space?
* what is the norm/metric/topology here for strong convergence?
* is the weak convergence here the same thing as weak-$\star$?

Then let's rewrite the below by summarizing [Higham D.J (2002)](http://homepages.warwick.ac.uk/~masdr/JOURNALPUBS/stuart51.pdf),
[this doctoral thesis](https://core.ac.uk/download/pdf/70597247.pdf)
and [this paper on Euler-Maryama](https://arxiv.org/abs/1610.07047.pdf)

In [ ]:
import torch

import numpy as np

<br>

Consider and SDE
$$
dX_t
    = \mu_t dt + \Sigma_t^{\tfrac12} dW_t
    \,, $$
where $\mu_t = \mu(X_t)$, $\Sigma_t = \Sigma(X_t)$ and $W_t$
is a multivaraite Brownian motion.

Euler-Maruyama numerical integration method of this SDE goes like this
(based on this lecture [on numerical methods in Computational Finance](https://www.mimuw.edu.pl/~apalczew/CFP_lecture5.pdf)).

The SDE is in fact the following integral equation
$$
X_t - X_0
    = \int_0^t \mu_\tau d\tau + \int_0^t \Sigma_\tau^{\tfrac12} dW_\tau
  \,, $$

so for $t$ and $t+\delta$ the Îto integral of the BM can be approximated
by the same finite difference, with which it is constructed (in $\ell_2$)

$$
\begin{align}
\int_t^{t+\delta} \Sigma_\tau^{\tfrac12} dW_\tau
    &\approx \Sigma_t^{\tfrac12} \bigl(W_{t+\delta} - W_t\bigr)
    \,, \\
\int_t^{t+\delta} \mu_\tau d\tau
    & \approx \mu_t \delta
    \,,
\end{align}
$$
where approximations are in $\ell_2$ ($\|\cdot\|_2^2 = \mathbb{E}(\cdot)^2$)
sense as $\delta \to 0$. Therefore the $\delta$-difference obeys

$$
X_{t+\delta} - X_t
    \sim \mathcal{N}_d(\mu_t \delta, \Sigma_t \delta)
    \,. $$

Consider a partition $0 = t_0 < t_1 < \cdots < t_n = T$ with $
\max_{k=1}^n (t_{k+1} - t_k) \leq \delta
$
and denote by
$ X^\delta_t $ the piecewise linear interpolation of $(X_{t_n})_{n=0}^n$:
$$
X^\delta_t
%     = \sum_{k=1}^n
%         1_{[t_{k-1}, t_k)}(t) \Bigl(
%             X_{t_k} + \frac{t - t_k}{t_{k+1} - t_k} (X_{t_{k+1}} - X_{t_k})
%         \Bigr)
    = \sum_{k \colon t \in [t_{k-1}, t_k)}
        X_{t_k} + \frac{t - t_k}{t_{k+1} - t_k} (X_{t_{k+1}} - X_{t_k})
    \,. $$

A numerical scheme for a given SDE has strong convergence of order $\gamma$
if there exists $C_T$ (depending on the SDE and time $T$) such that

$$
\mathbb{E} \biggl(
    \sup_{t \in [0, T]} \bigl\| X_t - X^\delta_t \bigr\|_2^2
\biggr)^{\tfrac12}
    \leq C_T \delta^\gamma
    \,. $$

Another notion of convergence for SDEs (and in general in probability) is weak
(weak-$\star$ in functional analysis). A scheme for an SDE has weak convergence
of order $\gamma$ if for any bounded continuous function $
    g \in C^\infty(
        \mathbb{R}^d \to \mathbb{R}
    )
$ there is a $C_{T,g}$ such that

$$
\bigl\lvert
    \mathbb{E} g(X_T) - g(X^\delta_T)
\bigr\rvert
    \leq C_{T,g} \delta^\gamma
    \,. $$
Note that weak convergence concerns the distribution at $T$ only.

<br>

In [ ]:
import torch

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import math

def euler_maruyama(mu, sigma, x_0, *, delta=1e-4):
    X_t = x_0.clone().detach()
    *head, _ = X_t.shape

    mu_t, sigma_t = mu(X_t), sigma(X_t)
    while True:
        X_t.add_(mu_t * delta)  # make a mul-copy of mu

        # torch.matmul(dW_t, torch.cholesky(sigma_t, upper=True))
        dW_t = torch.randn(*head, sigma_t.shape[-1], 1)
        dW_t.mul_(math.sqrt(delta))

        X_t.add_(torch.matmul(sigma_t, dW_t).squeeze(-1))

        yield X_t.clone()

        mu_t, sigma_t = mu(X_t), sigma(X_t)

In [ ]:
a = torch.randn(2, 20)
sigma_t = torch.cholesky(torch.mm(a, a.t()) * 1e-2, upper=False)
mu_t = torch.zeros(5, 2) * 2

In [ ]:
paths = [torch.randn(1, 2).repeat(5, 1) * 1e-1]
# paths = [torch.randn(5, 2) * 1e-1]
int_ = euler_maruyama(lambda x: mu_t, lambda x: sigma_t,
                      paths[-1], delta=1e-3)

paths.extend(X_t for _, X_t in zip(range(200), int_))

paths = torch.stack(paths, dim=1)

In [ ]:
fig = plt.figure(figsize=(14, 14))
ax = fig.add_subplot(111)

colors = plt.cm.Accent(np.linspace(0, 1, num=len(paths)))
for path, col in zip(paths, colors):
    path = path.numpy()
    xy, uv = path[:-1], path[1:] - path[:-1]
    ax.quiver(xy[:, 0], xy[:, 1], uv[:, 0], uv[:, 1], color=col,
              angles="xy", units="xy", scale=1., scale_units="xy")

In [ ]:
assert torch.allclose(torch.cholesky(a, upper=True).t(), torch.cholesky(a))